In [1]:
# | export
import numpy as np
import pandas as pd

In [2]:
#| default_exp sequences_preprocess

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

# Digestion profile per unique sequence
> functions to build the digestion profile per unique sequence

In [4]:
#| export
def count_sequences_from_mother(seqcell, mother_seq, n_sequences, mnase_array, population, aggregation_per_cell='mean'):
    """
    Return digestion mother sequence.
    columns: digestion from right to left
    rows: digestion from left to right
    """
    seqcell = seqcell[(seqcell['sort_population']==population)]
    mnase_array.sort()
    count_matrices_list = []

    row_name = []
    column_name = []
    
    for mnase in mnase_array:
        count_m = np.zeros((n_sequences, n_sequences))
        df = seqcell[(seqcell['units_mnase'] == mnase)]

        for i in range(0, count_m.shape[0]):
            seq = mother_seq[:-(i)] if i != 0 else mother_seq
            row_name.append(seq)
            for j in range(count_m.shape[1]):
                seqj = seq[j:]
                counts = df.loc[df['seq']==seqj, 'counts'].aggregate(aggregation_per_cell) #sum or average counts per cell
                count_m[i, j] = counts
                column_name.append(seqj)
        count_m[np.isnan(count_m)] = 0
        count_matrices_list.append(count_m)
    count_matrix_stacked = np.stack(count_matrices_list) # 
    return count_matrix_stacked, row_name, column_name

In [5]:
mnase_df = pd.DataFrame([1,2,3], columns=['units_mnase'])
mnase_df['counts'] = 1
sequences_df = pd.DataFrame(['ABCDE','ABC', 'BCD', 'CDE', 'CD'], columns=['seq'])
sequences_df['counts'] = 1
df = pd.merge(sequences_df, mnase_df, on='counts')
df['sort_population'] = 2

m, rows, columns = count_sequences_from_mother(df, 'ABCDE', 3, mnase_df['units_mnase'].values, 2, aggregation_per_cell='mean' )

# matrix:
#    | ABCDE BCDE CDE |
#    | ABCD  BCD  CD  | x mnases
#    | ABC   BC   C   |

assert (m[0,:,:] == np.array([[1, 0, 1], [0, 1, 1], [1, 0, 0]])).all()

In [6]:
#| export
def digestion_profile_from_count_matrix(count_matrix_stacked, mnases, digestion_direction, mother_seq, sequence_as_index=False):
    """
    digestion dataframe of a sequence.
    digestion_direction = 1: digestion by start (5'-->3') 
    digestion_direction = 2 : digestion by end (3' --> 5')
    """
    sum_counts = count_matrix_stacked.sum(axis=digestion_direction)

    if digestion_direction == 2: # digestion by end
        sequences_digested = [mother_seq] + [mother_seq[:-i] for i in range(1,sum_counts.shape[1])]
        sequences_digested.sort(key=len, reverse=True)
    elif digestion_direction == 1: # digestion by start
        sequences_digested = [mother_seq[i:] for i in range(sum_counts.shape[1])]
        sequences_digested.sort(key=len, reverse=True)
    else:
        raise ValueError('Index out of range, choose 1 for 5-3 digestion (start) or 2 for 3-5 digestion (end)')
    mnase_seq_df = pd.DataFrame(sum_counts, columns=sequences_digested, index=mnases)
    if sequence_as_index:
        mnase_seq_df = mnase_seq_df.transpose()
    return mnase_seq_df

In [7]:
mother_seq = 'ABCDE'

dp = digestion_profile_from_count_matrix(m, mnase_df['units_mnase'].values, 1, mother_seq)
assert 1 == dp.loc[1, 'BCDE']

dp = digestion_profile_from_count_matrix(m, mnase_df['units_mnase'].values, 2, mother_seq)
assert 2 == dp.loc[1, 'ABCD']

In [8]:
dp

,ABCDE,ABCD,ABC
1,2.0,2.0,1.0
2,2.0,2.0,1.0
3,2.0,2.0,1.0


In [9]:
rows


['ABCDE', 'ABCD', 'ABC', 'ABCDE', 'ABCD', 'ABC', 'ABCDE', 'ABCD', 'ABC']

In [10]:
#| export
def save_matrix(save_dir, m, rows, columns, mother_seq, p):
    import os 

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    file = save_dir + f'/m_{mother_seq}' + f'_p{p}.npy'

    with open(file, 'wb') as f:
        np.save(f, m)

    file = save_dir + '/rows_name' + f'_{mother_seq[:5]}..{mother_seq[-5:]}' + f'_p{p}.txt'
    rows = [r+'\n' for r in rows]

    with open(file, 'w') as f:
        f.writelines(rows)

    file = save_dir + '/columns_name' + f'_{mother_seq[:5]}..{mother_seq[-5:]}' + f'_p{p}.txt'
    columns = [c+'\n' for c in columns]

    with open(file, 'w') as f:
        f.writelines(columns)




In [11]:
save_dir = './test_dir'
mother_seq = 'ABCDE'
p = 1

save_matrix(save_dir, m, rows, columns, mother_seq, p)

In [12]:
#| export

def load_file(file):
    m = np.load(file)

    file = save_dir + '/columns_name' + f'_{mother_seq[:5]}..{mother_seq[-5:]}' + f'_p{p}.txt'

    with open(file, 'r') as f:
        cols = [line.rstrip('\n') for line in f]

    file = save_dir + '/rows_name' + f'_{mother_seq[:5]}..{mother_seq[-5:]}' + f'_p{p}.txt'

    with open(file, 'r') as f:
        rows = [line.rstrip('\n') for line in f]
    return m, rows, cols

In [13]:

file = save_dir + f'/m_{mother_seq}' + f'_p{p}.npy'
m1, rows1, cols1 = load_file(file)

assert (m == m1).all()